In [11]:
import os
import re
import json
import numpy as np
import cv2
import tempfile
import whisper
import mediapipe as mp

from ASL_model import TexttoMPPoints


In [ ]:
os.listdir("asl")

In [12]:
model = TexttoMPPoints()

In [13]:
# Step 1: Extract audio and generate transcript from video using Whisper
def extract_audio_and_transcribe(video_path):
    print("Extracting audio and transcribing...")
    temp_audio_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name

    # Extract audio using FFmpeg (PyAV can't easily write raw WAV directly)
    os.system(f"ffmpeg -i \"{video_path}\" -vn -acodec pcm_s16le -ar 16000 -ac 1 \"{temp_audio_path}\" -y")

    model = whisper.load_model("base")
    result = model.transcribe(temp_audio_path)
    os.remove(temp_audio_path)

    # Return as list of tuples (start, end, text)
    transcript = [(seg['start'], seg['end'], seg['text']) for seg in result['segments']]
    return transcript

In [14]:
FILLER_WORDS = {
    "a", "an", "the",
    "is", "are", "am", "was", "were", "be", "being", "been",
    "and", "or", "but", "so", "of", "to", "in", "for", "on", "at", "by",
    "um", "uh", "like", "you", "know", "okay", "right", "i", "mean"
}

In [16]:
# 1) sanitize transcript words
def sanitize_word(w):
    """Lowercase, strip non‑alphanumerics, skip fillers."""
    w = w.lower().strip()
    w = re.sub(r"[^a-z0-9]", "", w)
    if not w or w in FILLER_WORDS:
        return ""
    return w

# 2) build per-word timings
def build_word_timings(transcript):
    word_times = []
    for start, end, text in transcript:
        raw = text.strip().split()
        dur = end - start
        for i, rw in enumerate(raw):
            w = sanitize_word(rw)
            if not w: 
                continue
            w_start = start + (i/len(raw))*dur
            w_end   = start + ((i+1)/len(raw))*dur
            word_times.append({"word": w, "start": w_start, "end": w_end})
    return word_times

# 3) load GT JSONs
def load_gt_landmarks(folder="landmark_data"):
    gt = {}
    for fn in os.listdir(folder):
        if not fn.endswith(".json"):
            continue
        w = fn[:-5]
        gt[w] = json.load(open(os.path.join(folder, fn)))
    return gt

# 4) precompute predictions
def load_pred_landmarks(words):
    preds = {}
    for raw in set(words):
        w = sanitize_word(raw)
        if not w: 
            continue
        preds[w] = model.predict_landmarks(w)
    return preds

mp_pose      = mp.solutions.pose
mp_hands     = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

POSE_CONNECTIONS     = mp_pose.POSE_CONNECTIONS
HAND_CONNECTIONS     = mp_hands.HAND_CONNECTIONS
FACE_CONNECTIONS     = mp_face_mesh.FACEMESH_TESSELATION

def create_avatar_figure_image(flm, width=300, height=300):
    """
    Build a 300×300 RGBA avatar:
     - Pose skeleton in white
     - Facial mesh in green
     - Left hand in magenta
     - Right hand in cyan
    """
    im = np.zeros((height, width, 4), np.uint8)

    def to_px(pt):
        return int(pt["x"]*width), int(pt["y"]*height)

    # draw a connection set with given color & list of points
    def draw_connections(points, connections, color, thickness=2):
        for a,b in connections:
            if a < len(points) and b < len(points):
                pa = to_px(points[a])
                pb = to_px(points[b])
                cv2.line(im, pa, pb, color, thickness, cv2.LINE_AA)

    # 1) Pose
    if "pose" in flm:
        draw_connections(
            flm["pose"],
            POSE_CONNECTIONS,
            color=(255,255,255,200),
            thickness=3
        )

    # 2) Face
    if "face" in flm:
        draw_connections(
            flm["face"],
            FACE_CONNECTIONS,
            color=(0,255,0,100),
            thickness=1
        )
        # draw key facial landmarks (eyes, lips) a bit more boldly?
        for idx in [33,133,362,263]:  # outer landmarks of both eyes
            cv2.circle(im, to_px(flm["face"][idx]), 2, (0,200,0,255), -1)

    # 3) Left hand
    if "left_hand" in flm and flm["left_hand"]:
        draw_connections(
            flm["left_hand"],
            HAND_CONNECTIONS,
            color=(255,0,255,200),
            thickness=2
        )
        # draw wrist/joints
        for p in flm["left_hand"]:
            cv2.circle(im, to_px(p), 2, (255,0,255,255), -1)

    # 4) Right hand
    if "right_hand" in flm and flm["right_hand"]:
        draw_connections(
            flm["right_hand"],
            HAND_CONNECTIONS,
            color=(0,255,255,200),
            thickness=2
        )
        for p in flm["right_hand"]:
            cv2.circle(im, to_px(p), 2, (0,255,255,255), -1)

    return im

# 6) alpha-blend a small RGBA overlay into `img` at (x,y)
def overlay_image_alpha(img, overlay, x, y):
    h, w = overlay.shape[:2]
    alpha = overlay[:,:,3] / 255.0
    inv = 1.0 - alpha
    for c in range(3):
        img[y:y+h, x:x+w, c] = (
            alpha * overlay[:,:,c] +
            inv   * img[y:y+h, x:x+w, c]
        ).astype(np.uint8)

In [10]:
# paths & prep
video_path = "sample_text_video.mp4"
transcript = extract_audio_and_transcribe(video_path)
word_times = build_word_timings(transcript)

gt_land   = load_gt_landmarks("landmark_data")
pred_land = load_pred_landmarks([wt["word"] for wt in word_times])

cap    = cv2.VideoCapture(video_path)
fps    = cap.get(cv2.CAP_PROP_FPS)
W, H   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out    = cv2.VideoWriter("comparison_with_avatar.mp4",
                         cv2.VideoWriter_fourcc(*"mp4v"), fps, (W,H))

Extracting audio and transcribing...


In [11]:
frame_i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    t = frame_i / fps
    wt = next((w for w in word_times if w["start"] <= t < w["end"]), None)
    if wt:
        rel = (t - wt["start"]) / (wt["end"] - wt["start"])
        w   = wt["word"]

        gt_seq = gt_land.get(w, [])
        pd_seq = pred_land.get(w, [])
        if gt_seq and pd_seq:
            i_gt = min(int(rel * len(gt_seq)), len(gt_seq)-1)
            i_pd = min(int(rel * len(pd_seq)), len(pd_seq)-1)

            ov_gt = create_avatar_figure_image(gt_seq[i_gt])
            ov_pd = create_avatar_figure_image(pd_seq[i_pd])

            overlay_image_alpha(frame, ov_gt,   0,    H-300)
            overlay_image_alpha(frame, ov_pd,   W-300, H-300)

    out.write(frame)
    frame_i += 1

cap.release()
out.release()
print("Done — comparison_with_avatar.mp4 generated.")

Done — comparison_with_avatar.mp4 generated.


Comparison with Original Video

In [17]:
asl_caps = {}
def get_asl_frame(word, rel):
    """
    Returns a single BGR frame from asl_videos/<word>.mp4
    at relative position `rel` in [0..1].
    """
    path = os.path.join("asl_videos", f"{word}.mp4")
    if word not in asl_caps:
        cap = cv2.VideoCapture(path)
        if not cap.isOpened():
            return None
        cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        asl_caps[word] = (cap, cnt)
    cap, cnt = asl_caps[word]
    idx = min(int(rel * cnt), cnt-1)
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
    ret, frm = cap.read()
    return frm if ret else None

# ——————————————————————————————————
# 4) overlay helper
# ——————————————————————————————————

def overlay_image_alpha(img, overlay, x, y):
    """Alpha-blend RGBA overlay onto BGR img at (x,y)."""
    h, w = overlay.shape[:2]
    alpha = overlay[:,:,3:] / 255.0
    inv = 1.0 - alpha
    for c in range(3):
        img[y:y+h, x:x+w, c] = (
            alpha[:,:,0]*overlay[:,:,c] +
            inv[:,:,0]*img[y:y+h, x:x+w, c]
        ).astype(np.uint8)

# ——————————————————————————————————
# 5) draw a bigger, bolder stickman (GT)
# ——————————————————————————————————

mp_pose      = mp.solutions.pose
mp_hands     = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh

POSE_CONNS = mp_pose.POSE_CONNECTIONS
HAND_CONNS = mp_hands.HAND_CONNECTIONS
FACE_CONNS = mp_face_mesh.FACEMESH_TESSELATION

def create_bold_stickman(flm, width=350, height=350):
    """
    RGBA canvas showing:
     - Pose skeleton in thick white
     - Facial mesh in bold green
     - Hands in thick magenta/cyan
    """
    im = np.zeros((height, width, 4), np.uint8)
    def P(idx_list, idx):
        pt = idx_list[idx]
        return int(pt["x"]*width), int(pt["y"]*height)

    # draw connections
    def draw(conns, pts, color, thick):
        for a,b in conns:
            if a < len(pts) and b < len(pts):
                cv2.line(im, P(pts, a), P(pts, b), color, thick, cv2.LINE_AA)

    # 1) Pose
    if "pose" in flm:
        draw(POSE_CONNS, flm["pose"], (255,255,255,230), thick=6)

    # 2) Face
    if "face" in flm:
        draw(FACE_CONNS, flm["face"], (0,255,0,180), thick=2)
        # highlight eyes/lips
        for i in (33,133,362,263):
            cv2.circle(im, P(flm["face"],i), 4, (0,200,0,255), -1)

    # 3) Left hand
    if flm.get("left_hand"):
        draw(HAND_CONNS, flm["left_hand"], (255,0,255,220), thick=4)
        for p in flm["left_hand"]:
            cv2.circle(im, (int(p["x"]*width), int(p["y"]*height)), 4, (255,0,255,255), -1)

    # 4) Right hand
    if flm.get("right_hand"):
        draw(HAND_CONNS, flm["right_hand"], (0,255,255,220), thick=4)
        for p in flm["right_hand"]:
            cv2.circle(im, (int(p["x"]*width), int(p["y"]*height)), 4, (0,255,255,255), -1)

    return im

In [18]:
video_path = "Video_1.mp4"
transcript = extract_audio_and_transcribe(video_path)
word_times = build_word_timings(transcript)
gt_land    = load_gt_landmarks("landmark_data")

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
W, H = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(
    "Video_1_overlay_silent.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    fps,
    (W,H)
)

Extracting audio and transcribing...


In [19]:
frame_i = 0
while True:
    ret, frame = cap.read()
    if not ret: break

    t = frame_i / fps
    wt = next((w for w in word_times if w["start"] <= t < w["end"]), None)
    if wt:
        rel = (t - wt["start"]) / (wt["end"] - wt["start"])
        w   = wt["word"]

        # 1) draw GT stickman
        seq = gt_land.get(w, [])
        if seq:
            idx = min(int(rel * len(seq)), len(seq)-1)
            ov = create_bold_stickman(seq[idx])
            overlay_image_alpha(frame, ov, 0, H - ov.shape[0])

        # 2) paste ASL video frame
        asl_fr = get_asl_frame(w, rel)
        if asl_fr is not None:
            small = cv2.resize(asl_fr, (350,350), interpolation=cv2.INTER_AREA)
            frame[H-350:H, W-350:W] = small

    out.write(frame)
    frame_i += 1

# cleanup
cap.release()
for cap_asl, _ in asl_caps.values():
    cap_asl.release()
out.release()

print("Done → Video_1_overlay_silent.mp4")

Done → Video_1_overlay_silent.mp4


In [20]:
from helper_functions import restore_audio

In [21]:
restore_audio("Video_1_overlay_silent.mp4", "Video_1.mp4", "Video_1_final.mp4")

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.76.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1920, 1080], 'bitrate': 19350, 'fps': 29.93, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 28.63, 'bitrate': 19352, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [1920, 1080], 'video_bitrate': 19350, 'video_fps': 29.93, 'video_duration': 28.63, 'video_n_frames': 856}
c:\Users\PC\anaconda3\envs\text_to_asl\lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i Video_1_overlay_silent.mp4 -loglevel error -f image2pipe -vf scale=192

MoviePy - Done.
MoviePy - Writing video Video_1_final.mp4



MoviePy - Done !
MoviePy - video ready Video_1_final.mp4
New video file created!
